In [131]:
# Importacao das libraries
import pandas as pd
import functions as main
import numpy as np
from scipy import stats
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

# Load Data

In [150]:
ciclo = 5

opencsv = pd.read_csv(rf"D:\OneDrive - Instituto Politecnico de Castelo Branco\002_Learning\001_Pos_Graduacao\Ciclo 5 - Projeto Final\ProjetoFinal-PosGraduacaoEngenhariaFinanceira\ciclo{ciclo}_df.csv", sep=",")
opencsv.set_index('Date', inplace=True)
opencsv.index = pd.to_datetime(opencsv.index)
opencsv = opencsv.asfreq(pd.infer_freq(opencsv.index))
opencsv = opencsv.astype(float)

In [151]:
df_model = opencsv.copy()
df_model.drop(columns=["BTC_USD"])

,BalanceSheet_global,BalanceSheet_usa,Mining_Difficulty,USD_OnChain_Volume,GDP_usa,HashRate,M2_Liquidity,EURUSD=X,DX-Y.NYB,SPY,...,L1.Nr_Transactions,L2.Nr_Transactions,L1.Nr_Addresses,L2.Nr_Addresses,L1.BTC_PageViews,L2.BTC_PageViews,L1.Crypto_PageViews,L2.Crypto_PageViews,L1.BTC_Supply,L2.BTC_Supply
Date,,,,,,,,,,,,,,,,,,,,,
2024-04-19,4.366186,6.869555,13.936456,9.976805,4.451313,8.810200,4.324515,0.027112,2.025920,2.693347,...,5.651752,5.542373,5.834892,5.808687,3.899109,3.831294,1.568202,1.505150,7.294044,7.293900
2024-04-26,4.366354,6.869375,13.936456,9.878186,4.451313,8.762017,4.319079,0.030584,2.025060,2.704687,...,5.594602,5.651752,5.861619,5.834892,3.906820,3.899109,1.431364,1.568202,7.294190,7.294044
2024-05-03,4.366311,6.867024,13.936456,9.736209,4.451313,8.752032,4.317384,0.030724,2.021313,2.707268,...,5.645763,5.594602,5.822382,5.861619,3.889862,3.906820,1.568202,1.431364,7.294261,7.294190
2024-05-10,4.366711,6.866489,13.919854,9.619169,4.451313,8.777668,4.319235,0.032827,2.022470,2.715306,...,5.646151,5.645763,5.729990,5.822382,3.722881,3.889862,1.230449,1.568202,7.294327,7.294261
2024-05-17,4.368243,6.863577,13.919854,9.894055,4.451313,8.806631,4.318299,0.036141,2.018908,2.722426,...,5.818460,5.646151,5.676332,5.729990,3.656673,3.722881,1.431364,1.230449,7.294392,7.294327
2024-05-24,4.368018,6.863297,13.926247,9.911841,4.451313,8.810200,4.318950,0.033933,2.020030,2.722418,...,5.758527,5.818460,5.711131,5.676332,3.715586,3.656673,1.447158,1.431364,7.294461,7.294392
2024-05-31,4.367917,6.862389,13.926247,9.963684,4.451313,8.792955,4.317900,0.034845,2.019822,2.720717,...,5.818460,5.758527,5.684225,5.711131,3.713491,3.715586,1.414973,1.447158,7.294534,7.294461
2024-06-07,4.368913,6.860679,13.922812,9.956462,4.451313,8.771546,4.321165,0.037205,2.020734,2.726151,...,5.818460,5.818460,5.723201,5.684225,3.690993,3.713491,1.462398,1.414973,7.294603,7.294534
2024-06-14,4.368830,6.860875,13.922812,9.889418,4.451313,8.726468,4.321787,0.030957,2.023458,2.733225,...,5.421574,5.818460,5.670525,5.723201,3.644537,3.690993,1.447158,1.462398,7.294670,7.294603


In [ ]:
df_model.isnull().sum()

## Matriz de Correlacoes


In [ ]:
test_corr = df_model.copy()
test_corr = test_corr.iloc[:, :18] # apenas as variaveis originais
test_corr.shape

In [ ]:
main.correlation_map(df=test_corr, threshold=0.3)

## Seleção de Variáveis

Considerar técnicas como regressão lasso, ridge ou elastic net, que fazem seleção de variáveis automaticamente, pode ser útil se você suspeitar que nem todas as 20 variáveis são relevantes.

In [ ]:
df_model.columns

<div class="alert alert-block alert-info"><b>Hipoteses a testar</b>

H1: Ceteris paribus, Bitcoin é um ativo de especulação.

H2: Ceteris paribus, a performance de Bitcoin é influenciada por fatores macroeconómicos.

H3: Ceteris paribus, quanto mais liquidez há no mercado, maior é a apreciação de preço de Bitcoin.

H4: Ceteris paribus, os fatores endógenos de Bitcoin são os maiores explicadores da performance de Bitcoin.
</div>

## Regressao Linear Multipla

In [145]:
# Yt = b0 + b1X1t + b2X2t + εt

#   L1.
#   L2.

# Modelos endogenos Bitcoin
#   'Mining_Difficulty', 'USD_OnChain_Volume', 'HashRate', 'Miners_Revenue_USD', 
#   'Nr_Tx_Block', 'Nr_Transactions', 'Nr_Addresses', 'BTC_Supply'
model1a = ['Mining_Difficulty', 'USD_OnChain_Volume', 'BTC_Supply']
model1b = ['Nr_Transactions', 'L1.USD_OnChain_Volume', 'L1.BTC_Supply']
model1c = ['Nr_Addresses', 'Mining_Difficulty', 'Nr_Tx_Block']
model1d = ['Nr_Tx_Block', 'Nr_Addresses', 'Mining_Difficulty']


# Modelos Macro Economicos
#   'GDP_usa', 'EURUSD=X', 'DX-Y.NYB', 'SPY', 'QQQ', 'IWDA.AS', 'GLD', 'TLT', '^TNX', '^VIX'
#   'BalanceSheet_global', 'BalanceSheet_usa', '^TNX', 'M2_Liquidity', 'CL=F'
model2a = ['EURUSD=X', 'L2.SPY', 'TLT', 'CL=F']
model2b = ['L2.QQQ', 'GLD', 'L2.BalanceSheet_global', 'L2.CL=F']
model2c = ['GDP_usa', 'L1.DX-Y.NYB', 'L1.^VIX', 'L1.CL=F']
model2d = ['L2.M2_Liquidity', '^VIX', '^TNX']
model2e = ['IWDA.AS', 'L2.CL=F', 'L2.^TNX', 'L2.DX-Y.NYB', 'BalanceSheet_usa']#


# Modelos Attractiveness
#   'BTC_PageViews', 'Crypto_PageViews', 'L1.BTC_USD'
model3a = [  'BTC_PageViews', 'Crypto_PageViews', '^VIX', 'SPY', '^TNX']#
model3b = [ 'TLT', 'GLD', 'SPY', 'DX-Y.NYB', 'HashRate', 'Nr_Addresses']
model3c = [ 'L1.BTC_PageViews', 'L2.HashRate', 'L2.GLD', 'L1.Nr_Addresses']#

# Modelos gerais
#
'''model4a = []
model4b = []
model4c = []'''

X = df_model[model1b]
y = opencsv["BTC_USD"]

X = sm.add_constant(X) # Add a constant term to the predictors

model = sm.OLS(y, X)
results = model.fit() # Fit the OLS model

#results.summary() # Print the summary of the regression results

### Auto run Regressions

In [141]:
models_list = [model1a, model1b, model1c, model1d, model2a, model2b, 
               model2c, model2d, model2e, model3a, model3b, model3c]

models_columns = ['model_1a', 'model_1b', 'model_1c', 'model_1d', 'model_2a', 'model_2b', 
               'model_2c', 'model_2d', 'model_2e', 'model_3a', 'model_3b', 'model_3c']

def run_models_stats(df, models_list, y):
        count = 0
        for model_columns in models_list:
            X = df[model_columns]

            X = sm.add_constant(X)  # Add a constant term to the predictors

            model = sm.OLS(y, X)
            results = model.fit()  # Fit the OLS model
            
            print("\n##################################################")
            print("\nModel:", models_columns[count]) # columns name
            print("\nR-Squared:", results.rsquared_adj)
            #print("Variable:", results.params.index) # set as index
            #print("\nCoeff:", results.params) # coefficients
            #print("\nP-Value:", results.pvalues) # p-values
            print("No. Obs.:", results.nobs)
            print("Cond. no.:", results.condition_number)
            count += 1
            print("\n##################################################")


def run_models(df, models_list, y):
    f = open(f"Ciclo{ciclo}.txt", "w")
    count = 0
    for model_columns in models_list:
        X = df[model_columns]

        X = sm.add_constant(X)  # Add a constant term to the predictors

        model = sm.OLS(y, X)
        results = model.fit()  # Fit the OLS model
        
        f.write("\n####################################################################################################### \n\n")
        f.write(models_columns[count])
        f.write(" \n")
        f.write(f"{results.summary()}")
        count += 1
        f.write("\n####################################################################################################### \n\n")
    
    f.close()

In [152]:
y = opencsv["BTC_USD"]
run_models(df_model, models_list, y)

In [ ]:
#run_models_stats(df_model, models_list, y)

### Exemplo Data Frame

In [ ]:
data_store = pd.DataFrame()
data_store.index = df_model.columns


models_columns = ['model1a', 'model1b', 'model1c', 'model1d', 'model2a', 'model2b', 
               'model2c', 'model2d', 'model2e', 'model3a', 'model3b', 'model3c']

# Add new columns with a default value (e.g., None or 0)
for model in models_columns:
    data_store[model] = None  # or some other default value

data_store

In [ ]:
# list of all params 
dir(results)

https://www.statsmodels.org/dev/examples/notebooks/generated/ols.html

# Testes

### Homocesdasticity

In [ ]:
import statsmodels.formula.api as smf

# https://www.geeksforgeeks.org/how-to-perform-a-breusch-pagan-test-in-python/

### Normality of Residuals

Assumption: The residuals should be approximately normally distributed.

Test:
Q-Q Plot: Compare the quantiles of residuals to a normal distribution. If residuals are normal, the points will lie approximately on the 45-degree line.

Shapiro-Wilk Test or Kolmogorov-Smirnov Test: Statistical tests for normality of residuals.

Transformation: If residuals are not normally distributed, consider transforming the dependent variable or using robust regression methods.

### Independence of Errors


Assumption: The residuals (errors) should be independent of each other.

Test:

Durbin-Watson Test: Detects the presence of autocorrelation in residuals, especially in time series data.

Plot Residuals vs. Time: For time series data, plot residuals against time to detect patterns or autocorrelation.

Transformation:
Add Lagged Variables: For time series, include lagged values of the dependent variable as predictors.

### No Omitted Variable Bias

Assumption: All relevant independent variables are included in the model.

Test:

Ramsey RESET Test: Checks for omitted variables by testing for non-linearity in the model.

Remedies:

Include Relevant Variables: Ensure all theoretically important variables are included in the model.
Use Model Selection Criteria: Such as AIC or BIC to guide variable selection.

## Teste de Multicolinearidade


Como você tem muitas variáveis, é importante verificar a multicolinearidade, especialmente porque as variáveis acumuladas podem estar altamente correlacionadas entre si. Testes como o VIF (Variance Inflation Factor) podem ajudar a identificar problemas.

Multicollinearity describes the state where the independent variables used in a study exhibit a strong relationship with each other. This can pose a problem in many cases as you would normally want your independent variables to be… independent.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

X = 

# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns

# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]

vif_data

#       Interpreting the VIF Values:
#       VIF < 5: Generally considered acceptable, indicating low to moderate multicollinearity.
#       VIF between 5 and 10: Indicates moderate to high multicollinearity, which may be problematic, but not necessarily requiring immediate removal.
#       VIF > 10: Strong multicollinearity, typically suggesting that the variable should be considered for removal or that further investigation is needed.